In [22]:
import re


In [23]:
import httpx, json
import time
msgid = None
msgstr = None
in_msgstr = False
translate_api = "http://127.0.0.1:8080/api/translate"

In [24]:
def translate(text):
    
    payload = {"text": text}
    payload = json.dumps(payload)
    r = httpx.post(url = translate_api, data = payload).json()
    task_id = r["task_id"]
    res = httpx.get(url = f"{translate_api}/{task_id}").json()
    while res["task_ready"] == False: 
        res = httpx.get(url = f"{translate_api}/{task_id}").json()
        time.sleep(1)
    translate_res = res["result"]
    return translate_res

    

In [25]:
with open('django.po', 'r', encoding='utf-8') as file:
    lines = file.readlines()
    i = 0
    waitupdate = []
    pattern1 = re.compile('msgstr ""')
    for line in lines:
        if pattern1.match(line):
            # print(i)
            waitupdate.append(i)
        i += 1
    
    print(len(waitupdate))
    waitupdate2 = []
    pattern2 = re.compile(r'msgid "[^\"]+"')
    for x in waitupdate:
        if pattern2.match(lines[x-1]):
            waitupdate2.append(x)
            print("成功匹配"+lines[x-1])
        else:
            print("匹配失败"+lines[x-1])
    
    print(len(waitupdate2))
    print(waitupdate2)


195
匹配失败msgid ""

成功匹配msgid "English"

成功匹配msgid "Simplified Chinese"

成功匹配msgid "appIndex"

成功匹配msgid "appObjectStorage"

成功匹配msgid "appUser"

成功匹配msgid "Profile"

成功匹配msgid "Profiles"

成功匹配msgid "Flashcard"

成功匹配msgid "Flashcard Set"

成功匹配msgid "Flashcard Sets"

成功匹配msgid "Flashcards"

成功匹配msgid "Progress"

成功匹配msgid "Progresses"

成功匹配msgid "FreeHTML"

成功匹配msgid "compHtml"

成功匹配msgid "compHtmls"

成功匹配msgid "Myblog"

成功匹配msgid "category"

成功匹配msgid "categories"

成功匹配msgid "tag"

成功匹配msgid "tags"

成功匹配msgid "blog"

成功匹配msgid "blogs"

成功匹配msgid "featured blog"

成功匹配msgid "featured blogs"

成功匹配msgid "contact"

成功匹配msgid "contacts"

成功匹配msgid "Account Inactive"

成功匹配msgid "This account is inactive."

成功匹配msgid "E-mail Addresses"

成功匹配msgid "The following e-mail addresses are associated with your account:"

成功匹配msgid "Verified"

成功匹配msgid "Unverified"

成功匹配msgid "Primary"

成功匹配msgid "Make Primary"

成功匹配msgid "Re-send Verification"

成功匹配msgid "Remove"

成功匹配msgid "Warning:"

匹配失败"an e-mail a

In [26]:
#根据waitupdate2取得msgid的值组成新列表
waittranslate = []
pattern3 = re.compile('msgid "(.*)"')
for i in waitupdate2:
    print(pattern3.match(lines[i-1]).group(1))
    waittranslate.append(pattern3.match(lines[i-1]).group(1))

English
Simplified Chinese
appIndex
appObjectStorage
appUser
Profile
Profiles
Flashcard
Flashcard Set
Flashcard Sets
Flashcards
Progress
Progresses
FreeHTML
compHtml
compHtmls
Myblog
category
categories
tag
tags
blog
blogs
featured blog
featured blogs
contact
contacts
Account Inactive
This account is inactive.
E-mail Addresses
The following e-mail addresses are associated with your account:
Verified
Unverified
Primary
Make Primary
Re-send Verification
Remove
Add E-mail Address
Add E-mail
Do you really want to remove the selected e-mail address?
Hello from %(site_name)s!
Please Confirm Your E-mail Address
In case you forgot, your username is %(username)s.
Password Reset E-mail
Confirm E-mail Address
Confirm
Change Password
Sign Out
Sign In
Sign Up
Or
Are you sure you want to sign out?
You cannot remove your primary e-mail address (%(email)s).
Confirmation e-mail sent to %(email)s.
You have confirmed %(email)s.
Removed e-mail address %(email)s.
Successfully signed in as %(name)s.
You hav

In [27]:
def batchtranslate(texts : list[str]):
    
    payload = {"text": texts}
    payload = json.dumps(payload)
    r = httpx.post(url = "http://127.0.0.1:8080/api/translate/batch/", data = payload).json()
    task_ids = r["task_ids"]
    time.sleep(1*60)
    translate_res = []
    for task_id in task_ids: 
        res = httpx.get(url = f"{translate_api}/{task_id}").json()
        translate_res = translate_res+res["result"]
    return translate_res


In [29]:
results = batchtranslate(waittranslate)

In [34]:
#将results输出为json文件

json_str = json.dumps(results, indent=4, separators=(',', ':'), ensure_ascii=False)
with open("file.json", 'w',encoding="utf-8") as json_file:
    json_file.write(json_str)


In [35]:
with open("file.json", 'r',encoding="utf-8") as json_file:
    data = json.load(json_file)

In [36]:
print(data)

[{'translation_text': '英文', 'text': 'English'}, {'translation_text': '简体中文', 'text': 'Simplified Chinese'}, {'translation_text': '主页app', 'text': 'appIndex'}, {'translation_text': '对象存储app', 'text': 'appObjectStorage'}, {'translation_text': '用户app', 'text': 'appUser'}, {'translation_text': '个人简介', 'text': 'Profile'}, {'translation_text': '个人简介', 'text': 'Profiles'}, {'translation_text': '闪卡', 'text': 'Flashcard'}, {'translation_text': '闪卡集', 'text': 'Flashcard Set'}, {'translation_text': '闪卡集', 'text': 'Flashcard Sets'}, {'translation_text': '闪卡', 'text': 'Flashcards'}, {'translation_text': '进展', 'text': 'Progress'}, {'translation_text': '进展', 'text': 'Progresses'}, {'translation_text': 'FreeHTML', 'text': 'FreeHTML'}, {'translation_text': 'HTML组件', 'text': 'compHtml'}, {'translation_text': 'HTML组件', 'text': 'compHtmls'}, {'translation_text': '博客', 'text': 'Myblog'}, {'translation_text': '类别', 'text': 'category'}, {'translation_text': '类别', 'text': 'categories'}, {'translation_text': '

In [37]:
print(waittranslate)

['English', 'Simplified Chinese', 'appIndex', 'appObjectStorage', 'appUser', 'Profile', 'Profiles', 'Flashcard', 'Flashcard Set', 'Flashcard Sets', 'Flashcards', 'Progress', 'Progresses', 'FreeHTML', 'compHtml', 'compHtmls', 'Myblog', 'category', 'categories', 'tag', 'tags', 'blog', 'blogs', 'featured blog', 'featured blogs', 'contact', 'contacts', 'Account Inactive', 'This account is inactive.', 'E-mail Addresses', 'The following e-mail addresses are associated with your account:', 'Verified', 'Unverified', 'Primary', 'Make Primary', 'Re-send Verification', 'Remove', 'Warning:', 'Add E-mail Address', 'Add E-mail', 'Do you really want to remove the selected e-mail address?', 'Hello from %(site_name)s!', 'Please Confirm Your E-mail Address', 'In case you forgot, your username is %(username)s.', 'Password Reset E-mail', 'Confirm E-mail Address', 'Confirm', 'Change Password', 'Sign Out', 'Sign In', 'Sign Up', 'Or', 'Are you sure you want to sign out?', 'You cannot remove your primary e-ma

In [39]:
data_text = [ obj.get("text") for obj in data]
print(data_text)

['English', 'Simplified Chinese', 'appIndex', 'appObjectStorage', 'appUser', 'Profile', 'Profiles', 'Flashcard', 'Flashcard Set', 'Flashcard Sets', 'Flashcards', 'Progress', 'Progresses', 'FreeHTML', 'compHtml', 'compHtmls', 'Myblog', 'category', 'categories', 'tag', 'tags', 'blog', 'blogs', 'featured blog', 'featured blogs', 'contact', 'contacts', 'Account Inactive', 'This account is inactive.', 'E-mail Addresses', 'The following e-mail addresses are associated with your account:', 'Verified', 'Unverified', 'Primary', 'Make Primary', 'Re-send Verification', 'Remove', 'Warning:', 'Add E-mail Address', 'Add E-mail', 'Do you really want to remove the selected e-mail address?', 'Hello from %(site_name)s!', 'Please Confirm Your E-mail Address', 'In case you forgot, your username is %(username)s.', 'Password Reset E-mail', 'Confirm E-mail Address', 'Confirm', 'Change Password', 'Sign Out', 'Sign In', 'Sign Up', 'Or', 'Are you sure you want to sign out?', 'You cannot remove your primary e-ma

In [40]:
print(waittranslate==data_text)

True


In [41]:
data_translate = [obj.get("translation_text") for obj in data]

In [43]:
len(waitupdate2) == len(data_translate)

True

In [44]:
print(lines)

['# SOME DESCRIPTIVE TITLE.\n', "# Copyright (C) YEAR THE PACKAGE'S COPYRIGHT HOLDER\n", '# This file is distributed under the same license as the PACKAGE package.\n', '# FIRST AUTHOR <EMAIL@ADDRESS>, YEAR.\n', '#\n', '#, fuzzy\n', 'msgid ""\n', 'msgstr ""\n', '"Project-Id-Version: PACKAGE VERSION\\n"\n', '"Report-Msgid-Bugs-To: \\n"\n', '"POT-Creation-Date: 2024-09-19 19:39+0800\\n"\n', '"PO-Revision-Date: YEAR-MO-DA HO:MI+ZONE\\n"\n', '"Last-Translator: FULL NAME <EMAIL@ADDRESS>\\n"\n', '"Language-Team: LANGUAGE <LL@li.org>\\n"\n', '"MIME-Version: 1.0\\n"\n', '"Content-Type: text/plain; charset=UTF-8\\n"\n', '"Content-Transfer-Encoding: 8bit\\n"\n', '"Plural-Forms: nplurals=1; plural=0;\\n"\n', '\n', '#: .\\MainConfig\\settings.py:190\n', 'msgid "English"\n', 'msgstr ""\n', '\n', '#: .\\MainConfig\\settings.py:191\n', 'msgid "Simplified Chinese"\n', 'msgstr ""\n', '\n', '#: .\\appIndex\\apps.py:8\n', 'msgid "appIndex"\n', 'msgstr ""\n', '\n', '#: .\\appObjectStorage\\apps.py:7\n', 'm

In [52]:
for i in range(len(waitupdate2)):
    lines[waitupdate2[i]] = f"msgstr \"{data_translate[i]}\" \n"

In [53]:
lines


['# SOME DESCRIPTIVE TITLE.\n',
 "# Copyright (C) YEAR THE PACKAGE'S COPYRIGHT HOLDER\n",
 '# This file is distributed under the same license as the PACKAGE package.\n',
 '# FIRST AUTHOR <EMAIL@ADDRESS>, YEAR.\n',
 '#\n',
 '#, fuzzy\n',
 'msgid ""\n',
 'msgstr ""\n',
 '"Project-Id-Version: PACKAGE VERSION\\n"\n',
 '"Report-Msgid-Bugs-To: \\n"\n',
 '"POT-Creation-Date: 2024-09-19 19:39+0800\\n"\n',
 '"PO-Revision-Date: YEAR-MO-DA HO:MI+ZONE\\n"\n',
 '"Last-Translator: FULL NAME <EMAIL@ADDRESS>\\n"\n',
 '"Language-Team: LANGUAGE <LL@li.org>\\n"\n',
 '"MIME-Version: 1.0\\n"\n',
 '"Content-Type: text/plain; charset=UTF-8\\n"\n',
 '"Content-Transfer-Encoding: 8bit\\n"\n',
 '"Plural-Forms: nplurals=1; plural=0;\\n"\n',
 '\n',
 '#: .\\MainConfig\\settings.py:190\n',
 'msgid "English"\n',
 'msgstr "英文" \n',
 '\n',
 '#: .\\MainConfig\\settings.py:191\n',
 'msgid "Simplified Chinese"\n',
 'msgstr "简体中文" \n',
 '\n',
 '#: .\\appIndex\\apps.py:8\n',
 'msgid "appIndex"\n',
 'msgstr "主页app" \n',
 '\n

In [54]:
with open('django.po', 'w', encoding='utf-8') as file:
    file.writelines(lines)